In [1]:
import pandas as pd
# Load the datasets
ratings = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
movies = pd.read_csv('u.item', sep='|', encoding='latin-1', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

In [2]:
movies=movies.drop('video_release_date',axis=1)
movies=movies.dropna()
movies = movies.reset_index(drop=True)

In [3]:
ratings=ratings[ratings['item_id']!=267]
ratings = ratings.reset_index(drop=True)

In [4]:
# Merge the ratings and movies datasets
merged_df = pd.merge(ratings, movies, left_on='item_id', right_on='movie_id')

# Display the first few rows of the merged dataset
merged_df.head()

,user_id,item_id,rating,timestamp,movie_id,movie_title,release_date,IMDb_URL,unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,242,Kolya (1996),24-Jan-1997,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,302,L.A. Confidential (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,0,0,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,377,Heavyweights (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Heavyweights%...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,51,Legends of the Fall (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?Legends%20of%...,0,0,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,346,Jackie Brown (1997),01-Jan-1997,http://us.imdb.com/M/title-exact?imdb-title-11...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'], unit='s')

In [9]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class Reviews:
    def __init__(self, df):
        self.movies = df
        self.url = "https://www.imdb.com/"
        self.tag_words = []
        self.reviews = {}

    def make_tagword(self):
        pattern = r'^(.*?)\(.*[^)](.{5})$'
        self.unique_movies = list(self.movies['movie_title'].unique())
        for tag in self.unique_movies:
            match = re.search(pattern, tag)
            if match:
                before_bracket = match.group(1)
                last_five = match.group(2)
                self.tag_words.append(before_bracket + " " + last_five)
            else:
                self.tag_words.append(tag)

    def initiate_driver(self):
        driver_path = 'C:/Users/Dell/Downloads/chromedriver-win64 (1)/chromedriver-win64/chromedriver.exe'  # Update with the correct path
        service = Service(driver_path)
        options = webdriver.ChromeOptions()
        options.headless = False  # Set to True if you want headless mode
        self.driver = webdriver.Chrome(service=service, options=options)
        self.wait = WebDriverWait(self.driver, 10)
        self.driver.get(self.url)

    def get_search_box(self):
        try:
            self.search_box = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "suggestion-search"))
            )
        except Exception as e:
            print(f"Failed to locate search box: {e}")
            self.search_box = None

    def make_search(self, tag):
        if not hasattr(self, 'search_box') or self.search_box is None:
            print("Search box is not initialized.")
            return False
        print(f"Searching for: {tag}")
        self.search_box.clear()
        self.search_box.send_keys(tag)
        self.search_box.send_keys(Keys.RETURN)
        try:
            link = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'ipc-metadata-list-summary-item__t'))
            )
            self.driver.execute_script("arguments[0].scrollIntoView();", link)
            self.driver.execute_script("arguments[0].click();", link)
        except Exception as e:
            print(f"Failed to find or click the movie link for {tag}: {e}")
            return False
        return True

    def click_reviews(self):
        try:
            user_review_links = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "/reviews")]'))
            )
            self.driver.execute_script("arguments[0].scrollIntoView();", user_review_links)
            self.driver.execute_script("arguments[0].click();", user_review_links)
        except Exception as e:
            print(f"Failed to click reviews: {e}")
            return False
        return True

    def get_reviews(self, tag):
        try:
            div_elements = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.text.show-more__control.clickable'))
            )
            reviews_text = [div.text for div in div_elements[:10]]
            self.reviews[tag] = reviews_text
        except Exception as e:
            print(f"Failed to get reviews for {tag}: {e}")

    def go_back(self):
        self.driver.back()

    def close_driver(self):
        self.driver.quit()

    def reviews_to_dataframe(self):
        reviews_dict = {}
        for movie, reviews in self.reviews.items():
            combined_reviews = " ".join(reviews)
            reviews_dict[movie] = combined_reviews

        reviews_df = pd.DataFrame(list(reviews_dict.items()), columns=['movie_title', 'reviews'])
        return reviews_df

    def save_dataframe_as_csv(self, df, filename='movies_with_reviews.csv'):
        df.to_csv(filename, index=False)


In [18]:
a= merged_df.drop_duplicates(subset='movie_title')
a = a.reset_index(drop=True)
a=a[['movie_title']]
a

,movie_title
0,Kolya (1996)
1,L.A. Confidential (1997)
2,Heavyweights (1994)
3,Legends of the Fall (1994)
4,Jackie Brown (1997)
...,...
1656,Mamma Roma (1962)
1657,"Eighth Day, The (1996)"
1658,Girls Town (1996)
1659,"Silence of the Palace, The (Saimt el Qusur) (1..."


In [19]:
# Initialize and run the review scraper
rev = Reviews(a)
rev.make_tagword()
rev.initiate_driver()

# IMDb review scraping
for tag in rev.tag_words:
    rev.get_search_box()  # Ensure search box is initialized
    if rev.make_search(tag):
        if rev.click_reviews():
            rev.get_reviews(tag)
        rev.go_back()

rev.close_driver()

# Add reviews to the DataFrame
reviews_df = rev.reviews_to_dataframe()
a_with_reviews = pd.merge(a, reviews_df, how='left', on='movie_title')

# Save DataFrame with reviews to CSV
rev.save_dataframe_as_csv(a_with_reviews)

print("DataFrame with reviews has been saved to 'movies_with_reviews.csv'.")

Searching for: Kolya (1996)
Searching for: L.A. Confidential (1997)
Searching for: Heavyweights (1994)
Searching for: Legends of the Fall (1994)
Searching for: Jackie Brown (1997)
Searching for: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
Searching for: Hunt for Red October, The (1990)
Searching for: Jungle Book, The (1994)
Searching for: Grease (1978)
Searching for: Remains of the Day, The (1993)
Searching for: Men in Black (1997)
Searching for: Romy and Michele's High School Reunion (1997)
Searching for: Star Trek: First Contact (1996)
Searching for: To Wong Foo, Thanks for Everything! Julie Newmar (1995)
Searching for: Batman Forever (1995)
Searching for: Only You (1994)
Searching for: Age of Innocence, The (1993)
Searching for: Sabrina (1995)
Searching for: Just Cause (1995)
Searching for: Endless Summer 2, The (1994)
Searching for: Man Without a Face, The (1993)
Searching for: Sabrina (1954)
Searching for: Die Hard (1988)
Searching for: Twister (199

Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00007FF7AE78E3C9]
	(No symbol) [0x00007FF7AE686FDA]
	(No symbol) [0x00007FF7AE6D822C]
	(No symbol) [0x00007FF7AE6D850C]
	(No symbol) [0x00007FF7AE71DCB7]
	(No symbol) [0x00007FF7AE6FCAAF]
	(No symbol) [0x00007FF7AE71B041]
	(No symbol) [0x00007FF7AE6FC813]
	(No symbol) [0x00007FF7AE6CA6E5]
	(No symbol) [0x00007FF7AE6CB021]
	GetHandleVerifier [0x00007FF7AE90F83D+1301229]
	GetHandleVerifier [0x00007FF7AE91BDB7+1351783]
	GetHandleVerifier [0x00007FF7AE912A03+1313971]
	GetHandleVerifier [0x00007FF7AE80DD06+245686]
	(No symbol) [0x00007FF7AE79758F]
	(No symbol) [0x00007FF7AE793804]
	(No symbol) [0x00007FF7AE793992]
	(No symbol) [0x00007FF7AE78A3EF]
	BaseThreadInitThunk [0x00007FF96B3C7374+20]
	RtlUserThreadStart [0x00007FF96C63CC91+33]

Search box is not initialized.
Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00

Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00007FF7AE78E3C9]
	(No symbol) [0x00007FF7AE686FDA]
	(No symbol) [0x00007FF7AE6D822C]
	(No symbol) [0x00007FF7AE6D850C]
	(No symbol) [0x00007FF7AE71DCB7]
	(No symbol) [0x00007FF7AE6FCAAF]
	(No symbol) [0x00007FF7AE71B041]
	(No symbol) [0x00007FF7AE6FC813]
	(No symbol) [0x00007FF7AE6CA6E5]
	(No symbol) [0x00007FF7AE6CB021]
	GetHandleVerifier [0x00007FF7AE90F83D+1301229]
	GetHandleVerifier [0x00007FF7AE91BDB7+1351783]
	GetHandleVerifier [0x00007FF7AE912A03+1313971]
	GetHandleVerifier [0x00007FF7AE80DD06+245686]
	(No symbol) [0x00007FF7AE79758F]
	(No symbol) [0x00007FF7AE793804]
	(No symbol) [0x00007FF7AE793992]
	(No symbol) [0x00007FF7AE78A3EF]
	BaseThreadInitThunk [0x00007FF96B3C7374+20]
	RtlUserThreadStart [0x00007FF96C63CC91+33]

Search box is not initialized.
Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00

Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00007FF7AE78E3C9]
	(No symbol) [0x00007FF7AE686FDA]
	(No symbol) [0x00007FF7AE6D822C]
	(No symbol) [0x00007FF7AE6D850C]
	(No symbol) [0x00007FF7AE71DCB7]
	(No symbol) [0x00007FF7AE6FCAAF]
	(No symbol) [0x00007FF7AE71B041]
	(No symbol) [0x00007FF7AE6FC813]
	(No symbol) [0x00007FF7AE6CA6E5]
	(No symbol) [0x00007FF7AE6CB021]
	GetHandleVerifier [0x00007FF7AE90F83D+1301229]
	GetHandleVerifier [0x00007FF7AE91BDB7+1351783]
	GetHandleVerifier [0x00007FF7AE912A03+1313971]
	GetHandleVerifier [0x00007FF7AE80DD06+245686]
	(No symbol) [0x00007FF7AE79758F]
	(No symbol) [0x00007FF7AE793804]
	(No symbol) [0x00007FF7AE793992]
	(No symbol) [0x00007FF7AE78A3EF]
	BaseThreadInitThunk [0x00007FF96B3C7374+20]
	RtlUserThreadStart [0x00007FF96C63CC91+33]

Search box is not initialized.
Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00

Searching for: Clueless (1995)
Searching for: Rudy (1993)
Searching for: Fan, The (1996)
Searching for: Seven  1995)
Searching for: Great Escape, The (1963)
Searching for: Jurassic Park (1993)
Searching for: Dumbo (1941)
Searching for: Four Rooms (1995)
Searching for: Kicking and Screaming (1995)
Searching for: Nikita  1990)
Searching for: Grand Day Out, A (1992)
Searching for: Addiction, The (1995)
Searching for: Singin' in the Rain (1952)
Searching for: Shawshank Redemption, The (1994)
Searching for: One Flew Over the Cuckoo's Nest (1975)
Searching for: Maltese Falcon, The (1941)
Searching for: People vs. Larry Flynt, The (1996)
Searching for: Ulee's Gold (1997)
Searching for: Field of Dreams (1989)
Searching for: MatchMaker, The (1997)
Searching for: Fled (1996)
Searching for: Back to the Future (1985)
Searching for: Kicked in the Head (1997)
Searching for: Desperado (1995)
Searching for: Glengarry Glen Ross (1992)
Searching for: Mother (1996)
Searching for: Alien: Resurrection (199

...

Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00007FF7AE78E3C9]
	(No symbol) [0x00007FF7AE686FDA]
	(No symbol) [0x00007FF7AE6D822C]
	(No symbol) [0x00007FF7AE6D850C]
	(No symbol) [0x00007FF7AE71DCB7]
	(No symbol) [0x00007FF7AE6FCAAF]
	(No symbol) [0x00007FF7AE71B041]
	(No symbol) [0x00007FF7AE6FC813]
	(No symbol) [0x00007FF7AE6CA6E5]
	(No symbol) [0x00007FF7AE6CB021]
	GetHandleVerifier [0x00007FF7AE90F83D+1301229]
	GetHandleVerifier [0x00007FF7AE91BDB7+1351783]
	GetHandleVerifier [0x00007FF7AE912A03+1313971]
	GetHandleVerifier [0x00007FF7AE80DD06+245686]
	(No symbol) [0x00007FF7AE79758F]
	(No symbol) [0x00007FF7AE793804]
	(No symbol) [0x00007FF7AE793992]
	(No symbol) [0x00007FF7AE78A3EF]
	BaseThreadInitThunk [0x00007FF96B3C7374+20]
	RtlUserThreadStart [0x00007FF96C63CC91+33]

Search box is not initialized.
Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00

Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00007FF7AE78E3C9]
	(No symbol) [0x00007FF7AE686FDA]
	(No symbol) [0x00007FF7AE6D822C]
	(No symbol) [0x00007FF7AE6D850C]
	(No symbol) [0x00007FF7AE71DCB7]
	(No symbol) [0x00007FF7AE6FCAAF]
	(No symbol) [0x00007FF7AE71B041]
	(No symbol) [0x00007FF7AE6FC813]
	(No symbol) [0x00007FF7AE6CA6E5]
	(No symbol) [0x00007FF7AE6CB021]
	GetHandleVerifier [0x00007FF7AE90F83D+1301229]
	GetHandleVerifier [0x00007FF7AE91BDB7+1351783]
	GetHandleVerifier [0x00007FF7AE912A03+1313971]
	GetHandleVerifier [0x00007FF7AE80DD06+245686]
	(No symbol) [0x00007FF7AE79758F]
	(No symbol) [0x00007FF7AE793804]
	(No symbol) [0x00007FF7AE793992]
	(No symbol) [0x00007FF7AE78A3EF]
	BaseThreadInitThunk [0x00007FF96B3C7374+20]
	RtlUserThreadStart [0x00007FF96C63CC91+33]

Search box is not initialized.
Failed to locate search box: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00

Searching for: Paradise Road (1997)
Searching for: Killer  1994)
Searching for: Bent (1997)
Searching for: Umbrellas of Cherbourg, The  1964)
Searching for: Contempt  1963)
Searching for: Killer: A Journal of Murder (1995)
Searching for: Journey of August King, The (1995)
Searching for: New Jersey Drive (1995)
Searching for: Jury Duty (1995)
Searching for: U.S. Marshalls (1998)
Searching for: Kim (1950)
Searching for: National Lampoon's Senior Trip (1995)
Searching for: Picture Bride (1995)
Failed to click reviews: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00007FF7AE78E3C9]
	(No symbol) [0x00007FF7AE686FDA]
	(No symbol) [0x00007FF7AE6D822C]
	(No symbol) [0x00007FF7AE6D850C]
	(No symbol) [0x00007FF7AE71DCB7]
	(No symbol) [0x00007FF7AE6FCAAF]
	(No symbol) [0x00007FF7AE71B041]
	(No symbol) [0x00007FF7AE6FC813]
	(No symbol) [0x00007FF7AE6CA6E5]
	(No symbol) [0x00007FF7AE6CB021]
	GetHandleVerifier [0x00007FF7AE90F83D+1301229]
	GetHandleVerifier [0x0

Searching for: Desert Winds (1995)
Failed to get reviews for Desert Winds (1995): Message: 

Searching for: Last Summer in the Hamptons (1995)
Searching for: Calendar Girl (1993)
Searching for: Being Human (1993)
Searching for: Broken English (1996)
Searching for: Witness (1985)
Searching for: American Dream (1990)
Searching for: Nemesis 2: Nebula (1995)
Searching for: Love Serenade (1996)
Searching for: Angel on My Shoulder (1946)
Searching for: Germinal (1993)
Searching for: Wedding Gift, The (1994)
Failed to get reviews for Wedding Gift, The (1994): Message: 

Searching for: Here Comes Cookie (1935)
Searching for: Grateful Dead (1995)
Failed to click reviews: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00007FF7AE78E3C9]
	(No symbol) [0x00007FF7AE686FDA]
	(No symbol) [0x00007FF7AE6D822C]
	(No symbol) [0x00007FF7AE6D850C]
	(No symbol) [0x00007FF7AE71DCB7]
	(No symbol) [0x00007FF7AE6FCAAF]
	(No symbol) [0x00007FF7AE71B041]
	(No symbol) [0x00007FF

Searching for: Ripe (1996)
Searching for: Ed's Next Move (1996)
Searching for: Two Friends  986) 
Searching for: Men of Means (1998)
Searching for: Niagara, Niagara (1997)
Searching for: Spirits of the Dead  1968)
Searching for: Glass Shield, The (1994)
Searching for: Slingshot, The (1993)
Searching for: Scarlet Letter, The (1926)
Searching for: Condition Red (1995)
Searching for: Mat' i syn (1997)
Searching for: Careful (1992)
Searching for: They Made Me a Criminal (1939)
Searching for: Marlene Dietrich: Shadow and Light  996) 
Failed to click reviews: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AE7D9632+30946]
	(No symbol) [0x00007FF7AE78E3C9]
	(No symbol) [0x00007FF7AE686FDA]
	(No symbol) [0x00007FF7AE6D822C]
	(No symbol) [0x00007FF7AE6D850C]
	(No symbol) [0x00007FF7AE71DCB7]
	(No symbol) [0x00007FF7AE6FCAAF]
	(No symbol) [0x00007FF7AE71B041]
	(No symbol) [0x00007FF7AE6FC813]
	(No symbol) [0x00007FF7AE6CA6E5]
	(No symbol) [0x00007FF7AE6CB021]
	GetHandleVerifier [0x00007FF7AE

In [22]:
reviews_df

,movie_title,reviews
0,Kolya (1996),What is a man to do who has resisted marriage ...
1,L.A. Confidential (1997),Give a collection of great actors a great stor...
2,Heavyweights (1994),"Despite being a child of the '90s, somehow I m..."
3,Legends of the Fall (1994),I have enjoyed watching the film a number of t...
4,Jackie Brown (1997),"Coming as it did after critical darlings ""Rese..."
...,...,...
1521,Mamma Roma (1962),"As my first Pasolini film, Mamma Roma is as go..."
1522,"Eighth Day, The (1996)","Everyone we meet influences our thinking, modi..."
1523,Girls Town (1996),"At the center of this largely improvised, some..."
1524,"Silence of the Palace, The 1994)",Young FBI Agent Clarice Starling is chasing a ...


In [23]:
reviews_df.to_csv('output1.csv')

In [24]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526 entries, 0 to 1525
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_title  1526 non-null   object
 1   reviews      1526 non-null   object
dtypes: object(2)
memory usage: 24.0+ KB


In [26]:
reviews_df.isnull().sum()

movie_title    0
reviews        0
dtype: int64